# Sequential/Compositional Function Calling with Gemini 

This example demonstrates sequential function calling with the Gemini. Sequential function calling describes a flow where the model needs to call multiple functions in order to complete a task, where the output of one function call is needed as input for the next function call. Sequential function calling is a form of compositional function calling and Gemini supports it using multiple turns of the conversation.

Example:
> " If it is warmer than 20°C, set the temperature to 20°C else set it to 18°C."

Here the model needs to call two functions:
1. Get the current temperature outside
2. Update the AC to match the current temperature

We are going to use the [automatic function calling](https://ai.google.dev/gemini-api/docs/function-calling?example=weather#automatic_function_calling_python_only) feature of Gemini SDK.  The SDK automatically converts the Python function to declarations, handles the function call execution and response cycle for you.

In [ ]:
%pip install google-genai geopy requests

In [ ]:
from google import genai
from geopy.geocoders import Nominatim
import requests
from datetime import datetime

# create client
client = genai.Client()
model_id = "gemini-2.5-flash" # "gemini-2.5-pro"

# Function to get the weather forecast for a given location and date
geolocator = Nominatim(user_agent="weather-app")
def get_weather_forecast(location:str, date:str) -> dict:
    """Gets the weather forecast for a given location and date.

    Args:
        location: The city name, e.g. San Francisco.
        date: The date to get the weather forecast for, in YYYY-MM-DD format.

    Returns:
        A dictionary mapping hourly timestamps to temperatures for the given date,
        or a dictionary with an "error" key if the location is not found or an API error occurs.
    """
    print(f"API CALL: get_weather_forecast(location={location}, date={date})")
    location_data = geolocator.geocode(location)
    if location_data:
        try:
            response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={location_data.latitude}&longitude={location_data.longitude}&hourly=temperature_2m&start_date={date}&end_date={date}")
            data = response.json()
            # Check if 'hourly', 'time', and 'temperature_2m' keys exist before accessing them
            if "hourly" in data and "time" in data["hourly"] and "temperature_2m" in data["hourly"]:
                print("API RESPONSE: " + str({time: temp for time, temp in zip(data["hourly"]["time"], data["hourly"]["temperature_2m"])}), flush=True)
                return {time: temp for time, temp in zip(data["hourly"]["time"], data["hourly"]["temperature_2m"])}
            else:
                return {"error": "Unexpected response format from weather API."}
        except requests.exceptions.RequestException as e:
            return {"error": f"API request failed: {str(e)}"}
        except KeyError:
            return {"error": "Invalid data format received from weather API."}
        except Exception as e:
            return {"error": f"An unexpected error occurred: {str(e)}"}
    else:
        return {"error": "Location not found"}


# Mock Python implementation for setting thermostat temperature
def set_thermostat_temperature(temperature: int) -> dict:
    """Sets the thermostat to a desired temperature.

    Args:
        temperature: The desired temperature to set the thermostat to, in Celsius.

    Returns:
        A dictionary confirming the status of the operation, e.g., {"status": "success"}.
    """
    print(f"API CALL: set_thermostat_temperature(temperature={temperature})")
    # In a real app, this would interact with a thermostat API
    print("API RESPONSE: " + str({"status": "success"}))
    return {"status": "success"}

In [2]:
prompt = f"""User Context:
- name: Philipp
- location: London
- today: {datetime.now().strftime("%Y-%m-%d")}

User: If it is warmer than 20°C, set the temperature to 20°C else set it to 18°C."""

# Make the request
response = client.models.generate_content(
    model=model_id,
    contents=prompt,
    config={
        "tools": [get_weather_forecast, set_thermostat_temperature]
    },
)

print(response.text) 

API CALL: get_weather_forecast(location=London, date=2025-09-21)
API RESPONSE: {'2025-09-21T00:00': 12.8, '2025-09-21T01:00': 12.2, '2025-09-21T02:00': 11.9, '2025-09-21T03:00': 11.4, '2025-09-21T04:00': 11.1, '2025-09-21T05:00': 11.1, '2025-09-21T06:00': 11.0, '2025-09-21T07:00': 11.1, '2025-09-21T08:00': 11.5, '2025-09-21T09:00': 12.3, '2025-09-21T10:00': 13.2, '2025-09-21T11:00': 13.9, '2025-09-21T12:00': 15.1, '2025-09-21T13:00': 15.4, '2025-09-21T14:00': 15.8, '2025-09-21T15:00': 15.7, '2025-09-21T16:00': 15.7, '2025-09-21T17:00': 15.6, '2025-09-21T18:00': 14.9, '2025-09-21T19:00': 14.0, '2025-09-21T20:00': 12.8, '2025-09-21T21:00': 12.1, '2025-09-21T22:00': 11.6, '2025-09-21T23:00': 11.0}
API CALL: set_thermostat_temperature(temperature=18)
API RESPONSE: {'status': 'success'}
The thermostat has been set to 18°C.



In [3]:
for call in response.automatic_function_calling_history:
    part = call.parts[0]
    if part.text:
        print(f"Text: {part.text}")
    if part.function_call:
        print(f"Function Name: {part.function_call.name} Function Arguments: {part.function_call.args}")
    if part.function_response:
        print(f"Function Response: {part.function_response}")

print(f"final response: {response.text}")

Text: User Context:
- name: Philipp
- location: London
- today: 2025-09-21

User: If it is warmer than 20°C, set the temperature to 20°C else set it to 18°C.
Function Name: get_weather_forecast Function Arguments: {'date': '2025-09-21', 'location': 'London'}
Function Response: will_continue=None scheduling=None id=None name='get_weather_forecast' response={'result': {'2025-09-21T00:00': 12.8, '2025-09-21T01:00': 12.2, '2025-09-21T02:00': 11.9, '2025-09-21T03:00': 11.4, '2025-09-21T04:00': 11.1, '2025-09-21T05:00': 11.1, '2025-09-21T06:00': 11.0, '2025-09-21T07:00': 11.1, '2025-09-21T08:00': 11.5, '2025-09-21T09:00': 12.3, '2025-09-21T10:00': 13.2, '2025-09-21T11:00': 13.9, '2025-09-21T12:00': 15.1, '2025-09-21T13:00': 15.4, '2025-09-21T14:00': 15.8, '2025-09-21T15:00': 15.7, '2025-09-21T16:00': 15.7, '2025-09-21T17:00': 15.6, '2025-09-21T18:00': 14.9, '2025-09-21T19:00': 14.0, '2025-09-21T20:00': 12.8, '2025-09-21T21:00': 12.1, '2025-09-21T22:00': 11.6, '2025-09-21T23:00': 11.0}}
Funct